# Try to get all hydrogens around metal atoms about a conventional cell

In [2]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import defaultdict

# Count the amount of a specie in a cell

In [11]:
pmg_struct = Structure.from_file("./CaH6.cif")
pmg_struct.composition.to_data_dict
# {
#   'reduced_cell_composition': Comp: Li2 Mg1 H16,
#   'unit_cell_composition': defaultdict(float,
#              {'Li': 16.0, 'Mg': 8.0, 'H': 128.0}),
#   'reduced_cell_formula': 'Li2MgH16',
#   'elements': ['Li', 'Mg', 'H'],
#   'nelements': 3
# }
pmg_struct.composition.num_atoms            # get amounts of all atoms
pmg_struct.composition.alphabetical_formula # 'H128 Li16 Mg8'
pmg_struct.composition.anonymized_formula   # 'AB2C16'
pmg_struct.composition.iupac_formula        # 'Li16 Mg8 H128'
pmg_struct.composition.chemical_system      # 'H-Li-Mg'
pmg_struct.composition.elements             # [Element Li, Element Mg, Element H]
pmg_struct.composition.special_formulas     #{
                                            #     'LiO': 'Li2O2',
                                            #     'NaO': 'Na2O2',
                                            #     'KO': 'K2O2',
                                            #     'HO': 'H2O2',
                                            #     'CsO': 'Cs2O2',
                                            #     'RbO': 'Rb2O2',
                                            #     'O': 'O2',
                                            #     'N': 'N2',
                                            #     'F': 'F2',
                                            #     'Cl': 'Cl2',
                                            #     'H': 'H2'
                                            # }
pmg_struct.composition.get_el_amt_dict()                    # defaultdict(float, {'Li': 16.0, 'Mg': 8.0, 'H': 128.0})
pmg_struct.composition.get_integer_formula_and_factor()     # ('Li2MgH16', 8.0)
pmg_struct.composition.get_reduced_composition_and_factor() # (Comp: Li2 Mg1 H16, 8)


(Comp: Ca1 H6, 2)

# **一个小工具** 输入一个结构，输出其约化化学式，空间群对称性，文件名

In [ ]:
import sys
from pathlib import Path

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

path = Path(sys.argv[1])
file_name = path.name
struct = Structure.from_file(path)
name = struct.composition.get_integer_formula_and_factor()[0]
num = SpacegroupAnalyzer(struct).get_space_group_number()
spg_symbol = SpacegroupAnalyzer(struct).get_space_group_symbol()
print(f"{num}-{spg_symbol}-{name}")
print(f"{file_name}")

In [39]:
from pymatgen.core.structure import Structure
from pymatgen.analysis.chemenv.coordination_environments.structure_environments \
    import ChemicalEnvironments, StructureEnvironments, DetailedVoronoiContainer
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies \
    import SimplestChemenvStrategy

structure = Structure.from_file("./CaH6.cif")
structure
# se = StructureEnvironments(voronoi_container)
# sc = SimplestChemenvStrategy()
# sc.get_site_neighbors(structure)

Structure Summary
Lattice
    abc : 4.80719 4.80719 4.80719
 angles : 90.0 90.0 90.0
 volume : 111.08971759553498
      A : 4.80719 0.0 2.9435549231965825e-16
      B : 7.730551120299031e-16 4.80719 2.9435549231965825e-16
      C : 0.0 0.0 4.80719
PeriodicSite: Ca (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ca (2.4036, 2.4036, 2.4036) [0.5000, 0.5000, 0.5000]
PeriodicSite: H (1.2018, 0.0000, 2.4036) [0.2500, 0.0000, 0.5000]
PeriodicSite: H (3.6054, 0.0000, 2.4036) [0.7500, 0.0000, 0.5000]
PeriodicSite: H (2.4036, 1.2018, 0.0000) [0.5000, 0.2500, 0.0000]
PeriodicSite: H (2.4036, 3.6054, 0.0000) [0.5000, 0.7500, 0.0000]
PeriodicSite: H (0.0000, 2.4036, 1.2018) [0.0000, 0.5000, 0.2500]
PeriodicSite: H (0.0000, 2.4036, 3.6054) [0.0000, 0.5000, 0.7500]
PeriodicSite: H (0.0000, 1.2018, 2.4036) [0.0000, 0.2500, 0.5000]
PeriodicSite: H (0.0000, 3.6054, 2.4036) [0.0000, 0.7500, 0.5000]
PeriodicSite: H (1.2018, 2.4036, 0.0000) [0.2500, 0.5000, 0.0000]
PeriodicSite: H (3.6054,

In [ ]:
vc = DetailedVoronoiContainer(structure)
se = StructureEnvironments(vc)
# sc = SimplestChemenvStrategy()

In [49]:
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import NearNeighbors

s = Structure.from_file("./CaH6.cif")
s.structures_allowed = True
nn  = NearNeighbors()
print(s.all)
# graph = StructureGraph.with_local_env_strategy(structure, strategy=nn)
nn.get_bonded_structure(s)

AttributeError: 'Structure' object has no attribute 'all'

In [59]:
from pymatgen.analysis.local_env import VoronoiNN

vnn = VoronoiNN()
nninfo = vnn.get_all_nn_info(structure)
# vnn.get_all_voronoi_polyhedra(structure)
len(vnn.get_voronoi_polyhedra(structure, 0))

24